In [ ]:
'''
La derniere étape de notre projet est d'utiliser les modèles de machine learning choisi pour essayer de prédire 
les données de 2022 puis de comparer les écarts avec le reel.
'''

In [1]:
# Étape 1 : Preprocessing et feature engineering de la base de donnée 2022 pour la rendre exploitable.

# Import des modules et fichiers.
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

df_2022_full=pd.read_csv('../data/world-hapiness-report-2022.csv',sep=";")

# Suppression de toutes les lignes non relatives à 2022.
df_2022=df_2022_full[(df_2022_full['year']==2022)]

# Ajout des continents.
df_continents=pd.read_csv('../data/pays&continents.csv',sep=';')
df_2022=df_2022.merge(right=df_continents,on='Country name',how='outer')
df_2022.insert(1,'Regional indicator',df_2022.pop('Regional indicator'))

# Suppression des valeurs manquantes.
df_2022=df_2022.dropna()

# Suppression de la colonne des années.
df_2022=df_2022.drop(['year'],axis=1)

# Copie du jeux de données.
df_2022_continents = df_2022.copy()

# Séparation des variables explicatives et de la variable nom de pays de la variable cible.
X_test2022_continents=df_2022_continents.drop(['Life Ladder', 'Country name'],axis=1)
y_test2022=df_2022_continents['Life Ladder']

#On encode les continents de X_2022
X_test2022_continents=pd.get_dummies(X_test2022_continents,columns=['Regional indicator'])

# Normalisation des données.
scaler=MinMaxScaler()
cols=['Log GDP per capita','Social support','Healthy life expectancy at birth','Freedom to make life choices',
      'Generosity','Perceptions of corruption','Positive affect','Negative affect']
X_test2022_continents[cols]=scaler.fit_transform(X_test2022_continents[cols])

# Suppression de la colonne "Regional indicator_Central and Eastern Europe" pour correspondre à l'encodage des données
# fait par le OneHotEncoding.
X_test2022_continents=X_test2022_continents.drop(['Regional indicator_Central and Eastern Europe'],axis=1)

# X_test2022_continents et y_test2022 sont alors respectivements les jeux d'entraînement et de test.

In [2]:
# Étape 2 : Récupération des jeux d'entraînement et de test générés dans les notebooks précédents (données de 2006 
# à 2021).

# Import des données.
X_train=pd.read_csv('../data/X_train.csv')
X_test=pd.read_csv('../data/X_test.csv')
y_train=pd.read_csv('../data/y_train.csv')
y_test=pd.read_csv('../data/y_test.csv')

# Concaténation.
X_train2021=pd.concat([X_train,X_test],axis=0)
y_train2021=pd.concat([y_train,y_test],axis=0)

# Transofrmation du jeu y en tableau python.
y_train2021=y_train2021['Life Ladder'].values

# Normalisation des données.
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
cols=['Log GDP per capita','Social support','Healthy life expectancy at birth','Freedom to make life choices',
      'Generosity', 'Perceptions of corruption','Positive affect','Negative affect']
X_train2021[cols]=scaler.fit_transform(X_train2021[cols])

In [3]:
# Étape 3: Faire tourner le modele de Random forest pour la regression et le tester.

from sklearn.ensemble import RandomForestRegressor
rf_model_continents = RandomForestRegressor(max_depth=30,max_features='sqrt',min_samples_leaf=1,min_samples_split=2,n_estimators=300)

rf_model_continents.fit(X_train2021,y_train2021)

print("Score sur jeu d'entrainement 2021 :",rf_model_continents.score(X_train2021,y_train2021))
print("Score sur jeu de test 2022 :",rf_model_continents.score(X_test2022_continents,y_test2022))

Score sur jeu d'entrainement 2021 : 0.9857439421804151
Score sur jeu de test 2022 : 0.7969173019970263


In [4]:
# Réalisation d'une prédiction du Life Ladder de 2022 par pays grâce au modèle Random forest.
y_predic2022_continents = rf_model_continents.predict(X_test2022_continents)

# Arrondissement de la prédictions à 2 décimales.
import numpy as np
y_predic2022_continents=np.round(y_predic2022_continents,2)

# Ajout d'une colonne avec les prédictions au jeu de données df_2022.
df_2022_continents['Predictions']=y_predic2022_continents
df_2022_continents=df_2022_continents.drop(cols,axis=1)

# Ajout d'une colonne pour représenter les variations en pourcentages.
df_2022_continents['Ecart en %']=np.round((df_2022_continents['Predictions']-df_2022_continents['Life Ladder'])/df_2022_continents['Life Ladder']*100,1)

In [5]:
df_2022_continents.head(10)

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
1,Albania,Central and Eastern Europe,5.21,4.95,-5.0
2,Argentina,Latin America and Caribbean,6.26,6.14,-1.9
3,Armenia,Commonwealth of Independent States,5.38,4.99,-7.2
4,Australia,North America and ANZ,7.04,7.00,-0.6
5,Austria,Western Europe,7.00,6.71,-4.1
6,Bangladesh,South Asia,3.41,4.03,18.2
7,Belgium,Western Europe,6.86,7.03,2.5
8,Benin,Sub-Saharan Africa,4.22,3.90,-7.6
9,Bolivia,Latin America and Caribbean,5.93,5.28,-11.0
10,Botswana,Sub-Saharan Africa,3.44,4.65,35.2


In [6]:
df_2022_continents.drop('Country name',axis=1).groupby('Regional indicator').agg('mean').sort_values('Ecart en %',ascending=True)

# On constate que le modèle prédit plutot bien avec toutefois des écarts plus importants pour les valeurs extremes.

,Life Ladder,Predictions,Ecart en %
Regional indicator,,,
Southeast Asia,5.530000,5.077500,-7.375000
Latin America and Caribbean,6.211111,5.785000,-6.711111
Commonwealth of Independent States,5.592500,5.221250,-6.350000
East Asia,5.973333,5.623333,-6.000000
Central and Eastern Europe,6.044545,5.758182,-4.409091
North America and ANZ,6.907500,6.765000,-2.150000
Western Europe,6.893333,6.801667,-1.350000
South Asia,4.270000,4.130000,0.133333
Sub-Saharan Africa,4.289643,4.288571,2.532143


In [7]:
display(df_2022_continents.sort_values('Life Ladder',ascending=True).head(10))
display(df_2022_continents.sort_values('Life Ladder',ascending=True).tail(10))

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
61,Lebanon,Middle East and North Africa,2.35,4.01,70.6
93,Sierra Leone,Sub-Saharan Africa,2.56,3.58,39.8
21,Congo (Kinshasa),Sub-Saharan Africa,3.21,4.21,31.2
113,Zimbabwe,Sub-Saharan Africa,3.30,4.09,23.9
66,Malawi,Sub-Saharan Africa,3.36,3.73,11.0
6,Bangladesh,South Asia,3.41,4.03,18.2
10,Botswana,Sub-Saharan Africa,3.44,4.65,35.2
19,Comoros,Sub-Saharan Africa,3.55,3.89,9.6
101,Tanzania,Sub-Saharan Africa,3.62,4.10,13.3
32,Ethiopia,Sub-Saharan Africa,3.63,4.37,20.4


,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
63,Lithuania,Central and Eastern Europe,7.04,5.91,-16.1
22,Costa Rica,Latin America and Caribbean,7.08,6.65,-6.1
64,Luxembourg,Western Europe,7.23,6.91,-4.4
83,Norway,Western Europe,7.30,7.40,1.4
78,Netherlands,Western Europe,7.39,7.06,-4.5
98,Sweden,Western Europe,7.43,7.45,0.3
45,Iceland,Western Europe,7.45,7.42,-0.4
26,Denmark,Western Europe,7.55,7.43,-1.6
50,Israel,Middle East and North Africa,7.66,6.80,-11.2
33,Finland,Western Europe,7.73,7.51,-2.8


In [8]:
display(df_2022_continents.sort_values('Ecart en %',ascending=False).head(10))
display(df_2022_continents.sort_values('Ecart en %',ascending=True).head(10))

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
61,Lebanon,Middle East and North Africa,2.35,4.01,70.6
93,Sierra Leone,Sub-Saharan Africa,2.56,3.58,39.8
10,Botswana,Sub-Saharan Africa,3.44,4.65,35.2
21,Congo (Kinshasa),Sub-Saharan Africa,3.21,4.21,31.2
113,Zimbabwe,Sub-Saharan Africa,3.30,4.09,23.9
32,Ethiopia,Sub-Saharan Africa,3.63,4.37,20.4
6,Bangladesh,South Asia,3.41,4.03,18.2
101,Tanzania,Sub-Saharan Africa,3.62,4.10,13.3
66,Malawi,Sub-Saharan Africa,3.36,3.73,11.0
39,Ghana,Sub-Saharan Africa,4.19,4.64,10.7


,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
20,Congo (Brazzaville),Sub-Saharan Africa,5.81,4.35,-25.1
77,Nepal,South Asia,5.47,4.26,-22.1
42,Guinea,Sub-Saharan Africa,5.32,4.19,-21.2
71,Mexico,Latin America and Caribbean,7.04,5.87,-16.6
112,Vietnam,Southeast Asia,6.27,5.25,-16.3
63,Lithuania,Central and Eastern Europe,7.04,5.91,-16.1
80,Nicaragua,Latin America and Caribbean,6.39,5.40,-15.5
30,El Salvador,Latin America and Caribbean,6.49,5.50,-15.3
92,Senegal,Sub-Saharan Africa,4.91,4.17,-15.1
73,Mongolia,East Asia,5.79,4.97,-14.2


In [9]:
'''
On pourrait se demander si lorsqu'il y a des valeurs un peu plus extremes, le continent n'entraîne pas le modele 
vers une réponse plus cohérente en fonction du continents. 
On réalise donc un essai sur les mêmes données amputées de la variable continent.
'''

"\nOn pourrait se demander si lorsqu'il y a des valeurs un peu plus extremes, le continent n'entraîne pas le modele \nvers une réponse plus cohérente en fonction du continents. \nOn réalise donc un essai sur les mêmes données amputées de la variable continent.\n"

In [10]:
# Séparation des variables explicatives de la variable cible et suppression des noms de pays.
X_test2022_no_continents=df_2022.drop(['Life Ladder','Country name','Regional indicator'],axis=1)
y_test2022_no_continents=df_2022['Life Ladder']

#On normalise les variables explicatives
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
cols=['Log GDP per capita','Social support','Healthy life expectancy at birth','Freedom to make life choices','Generosity',
      'Perceptions of corruption','Positive affect','Negative affect']
X_test2022_no_continents[cols]=scaler.fit_transform(X_test2022_no_continents[cols])

#X_test2022_no_continents et y_test2022_no_continents sont alors respectivements nos variables explicatives et notre variable cible de notre jeu de test

In [11]:
# Suppression variables des continents du jeu de données 2021.
X_train2021_no_continents=X_train2021.drop(["Regional indicator_Commonwealth of Independent States"
                              ,"Regional indicator_East Asia"
                              ,"Regional indicator_Latin America and Caribbean"
                              ,"Regional indicator_Middle East and North Africa"
                              ,"Regional indicator_North America and ANZ"
                              ,"Regional indicator_South Asia"
                              ,"Regional indicator_Southeast Asia"
                              ,"Regional indicator_Sub-Saharan Africa"
                              ,"Regional indicator_Western Europe"],axis=1)

#On normalise les variables explicatives
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
cols=['Log GDP per capita','Social support','Healthy life expectancy at birth','Freedom to make life choices','Generosity',
      'Perceptions of corruption','Positive affect','Negative affect']
X_train2021_no_continents[cols]=scaler.fit_transform(X_train2021_no_continents[cols])

In [12]:
from sklearn.ensemble import RandomForestRegressor
rf_model_no_continents = RandomForestRegressor(max_depth=30,max_features='sqrt',min_samples_leaf=1,min_samples_split=2,n_estimators=300)

rf_model_no_continents.fit(X_train2021_no_continents,y_train2021)

print("Score sur jeu d'entrainement 2021 :",rf_model_no_continents.score(X_train2021_no_continents,y_train2021))
print("Score sur jeu de test 2022 :",rf_model_no_continents.score(X_test2022_no_continents,y_test2022))

Score sur jeu d'entrainement 2021 : 0.9834139375479372
Score sur jeu de test 2022 : 0.7412677808451422


In [13]:
# Réalisation d'une prédiction du Life Ladder de 2022 par pays grâce au modèle Random forest.
y_predic2022_no_continents = rf_model_no_continents.predict(X_test2022_no_continents)

# Arrondissement de la prédictions à 2 décimales.
import numpy as np
y_predic2022_no_continents=np.round(y_predic2022_no_continents,2)

# Copie du jeu de donnée 2022 et suppression des continents.
df_2022_no_continents = df_2022.copy()

# Ajout d'une colonne avec les prédictions au jeu de données df_2022.
df_2022_no_continents['Predictions']=y_predic2022_no_continents
df_2022_no_continents=df_2022_no_continents.drop(cols,axis=1)

# Ajout d'une colonne pour représenter les variations en pourcentages.
df_2022_no_continents['Ecart en %']=np.round((df_2022_no_continents['Predictions']-df_2022_no_continents['Life Ladder'])/df_2022_no_continents['Life Ladder']*100,1)

In [14]:
df_2022_no_continents

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
1,Albania,Central and Eastern Europe,5.21,4.95,-5.0
2,Argentina,Latin America and Caribbean,6.26,6.08,-2.9
3,Armenia,Commonwealth of Independent States,5.38,4.91,-8.7
4,Australia,North America and ANZ,7.04,6.90,-2.0
5,Austria,Western Europe,7.00,6.68,-4.6
...,...,...,...,...,...
108,United States,North America and ANZ,6.69,6.06,-9.4
109,Uruguay,Latin America and Caribbean,6.67,6.30,-5.5
110,Uzbekistan,Commonwealth of Independent States,6.02,5.19,-13.8
112,Vietnam,Southeast Asia,6.27,5.30,-15.5


In [15]:
df_2022_no_continents.drop('Country name',axis=1).groupby('Regional indicator').agg('mean').sort_values('Ecart en %',ascending=True)

#Grâce à ce df on s'aperçoit que notre modele prédit plutot bien, il a un peu plus de mal a prédire les valeurs extremes

,Life Ladder,Predictions,Ecart en %
Regional indicator,,,
Latin America and Caribbean,6.211111,5.537222,-10.744444
Commonwealth of Independent States,5.592500,5.093750,-8.675000
Southeast Asia,5.530000,5.120000,-6.525000
East Asia,5.973333,5.616667,-6.100000
Central and Eastern Europe,6.044545,5.774545,-4.145455
North America and ANZ,6.907500,6.660000,-3.650000
Western Europe,6.893333,6.777222,-1.694444
South Asia,4.270000,4.143333,0.466667
Sub-Saharan Africa,4.289643,4.275714,2.028571


In [16]:
display(df_2022_no_continents.sort_values('Life Ladder',ascending=True).head(10))
display(df_2022_no_continents.sort_values('Life Ladder',ascending=True).tail(10))

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
61,Lebanon,Middle East and North Africa,2.35,4.02,71.1
93,Sierra Leone,Sub-Saharan Africa,2.56,3.41,33.2
21,Congo (Kinshasa),Sub-Saharan Africa,3.21,4.06,26.5
113,Zimbabwe,Sub-Saharan Africa,3.30,4.11,24.5
66,Malawi,Sub-Saharan Africa,3.36,3.72,10.7
6,Bangladesh,South Asia,3.41,4.01,17.6
10,Botswana,Sub-Saharan Africa,3.44,4.61,34.0
19,Comoros,Sub-Saharan Africa,3.55,3.89,9.6
101,Tanzania,Sub-Saharan Africa,3.62,4.22,16.6
32,Ethiopia,Sub-Saharan Africa,3.63,4.25,17.1


,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
63,Lithuania,Central and Eastern Europe,7.04,5.89,-16.3
22,Costa Rica,Latin America and Caribbean,7.08,6.67,-5.8
64,Luxembourg,Western Europe,7.23,6.85,-5.3
83,Norway,Western Europe,7.30,7.39,1.2
78,Netherlands,Western Europe,7.39,7.06,-4.5
98,Sweden,Western Europe,7.43,7.38,-0.7
45,Iceland,Western Europe,7.45,7.42,-0.4
26,Denmark,Western Europe,7.55,7.33,-2.9
50,Israel,Middle East and North Africa,7.66,6.49,-15.3
33,Finland,Western Europe,7.73,7.43,-3.9


In [17]:
display(df_2022_no_continents.sort_values('Ecart en %',ascending=False).head(10))
display(df_2022_no_continents.sort_values('Ecart en %',ascending=True).head(10))

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
61,Lebanon,Middle East and North Africa,2.35,4.02,71.1
10,Botswana,Sub-Saharan Africa,3.44,4.61,34.0
93,Sierra Leone,Sub-Saharan Africa,2.56,3.41,33.2
21,Congo (Kinshasa),Sub-Saharan Africa,3.21,4.06,26.5
113,Zimbabwe,Sub-Saharan Africa,3.30,4.11,24.5
6,Bangladesh,South Asia,3.41,4.01,17.6
32,Ethiopia,Sub-Saharan Africa,3.63,4.25,17.1
101,Tanzania,Sub-Saharan Africa,3.62,4.22,16.6
39,Ghana,Sub-Saharan Africa,4.19,4.69,11.9
66,Malawi,Sub-Saharan Africa,3.36,3.72,10.7


,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
20,Congo (Brazzaville),Sub-Saharan Africa,5.81,4.32,-25.6
80,Nicaragua,Latin America and Caribbean,6.39,4.89,-23.5
71,Mexico,Latin America and Caribbean,7.04,5.40,-23.3
30,El Salvador,Latin America and Caribbean,6.49,5.01,-22.8
77,Nepal,South Asia,5.47,4.25,-22.3
42,Guinea,Sub-Saharan Africa,5.32,4.18,-21.4
43,Honduras,Latin America and Caribbean,5.93,4.75,-19.9
41,Guatemala,Latin America and Caribbean,6.15,4.95,-19.5
37,Georgia,Commonwealth of Independent States,5.29,4.33,-18.1
9,Bolivia,Latin America and Caribbean,5.93,4.90,-17.4


In [18]:
# Pour terminer, on aggrège les deux jeux de données.
df_2022 = df_2022_continents
df_2022

,Country name,Regional indicator,Life Ladder,Predictions,Ecart en %
1,Albania,Central and Eastern Europe,5.21,4.95,-5.0
2,Argentina,Latin America and Caribbean,6.26,6.14,-1.9
3,Armenia,Commonwealth of Independent States,5.38,4.99,-7.2
4,Australia,North America and ANZ,7.04,7.00,-0.6
5,Austria,Western Europe,7.00,6.71,-4.1
...,...,...,...,...,...
108,United States,North America and ANZ,6.69,6.13,-8.4
109,Uruguay,Latin America and Caribbean,6.67,6.33,-5.1
110,Uzbekistan,Commonwealth of Independent States,6.02,5.44,-9.6
112,Vietnam,Southeast Asia,6.27,5.25,-16.3


In [19]:
# Changement de nom des colonnes
new_names =  {'Predictions' : 'Prédictions avec continents',
              'Ecart en %'  : 'Écart en % avec continents'}

df_2022 = df_2022.rename(new_names, axis = 1)
df_2022

,Country name,Regional indicator,Life Ladder,Prédictions avec continents,Écart en % avec continents
1,Albania,Central and Eastern Europe,5.21,4.95,-5.0
2,Argentina,Latin America and Caribbean,6.26,6.14,-1.9
3,Armenia,Commonwealth of Independent States,5.38,4.99,-7.2
4,Australia,North America and ANZ,7.04,7.00,-0.6
5,Austria,Western Europe,7.00,6.71,-4.1
...,...,...,...,...,...
108,United States,North America and ANZ,6.69,6.13,-8.4
109,Uruguay,Latin America and Caribbean,6.67,6.33,-5.1
110,Uzbekistan,Commonwealth of Independent States,6.02,5.44,-9.6
112,Vietnam,Southeast Asia,6.27,5.25,-16.3


In [20]:
df_2022['Prédictions sans continents'] = df_2022_no_continents['Predictions']
df_2022['Écart en % sans continents'] = df_2022_no_continents['Ecart en %']
df_2022

,Country name,Regional indicator,Life Ladder,Prédictions avec continents,Écart en % avec continents,Prédictions sans continents,Écart en % sans continents
1,Albania,Central and Eastern Europe,5.21,4.95,-5.0,4.95,-5.0
2,Argentina,Latin America and Caribbean,6.26,6.14,-1.9,6.08,-2.9
3,Armenia,Commonwealth of Independent States,5.38,4.99,-7.2,4.91,-8.7
4,Australia,North America and ANZ,7.04,7.00,-0.6,6.90,-2.0
5,Austria,Western Europe,7.00,6.71,-4.1,6.68,-4.6
...,...,...,...,...,...,...,...
108,United States,North America and ANZ,6.69,6.13,-8.4,6.06,-9.4
109,Uruguay,Latin America and Caribbean,6.67,6.33,-5.1,6.30,-5.5
110,Uzbekistan,Commonwealth of Independent States,6.02,5.44,-9.6,5.19,-13.8
112,Vietnam,Southeast Asia,6.27,5.25,-16.3,5.30,-15.5


In [21]:
# Ajout d'une colonne pour représenter les différences de pourcentages.
df_2022['Différences valeurs écarts'] = df_2022['Prédictions sans continents']-df_2022['Prédictions avec continents']
df_2022

,Country name,Regional indicator,Life Ladder,Prédictions avec continents,Écart en % avec continents,Prédictions sans continents,Écart en % sans continents,Différences valeurs écarts
1,Albania,Central and Eastern Europe,5.21,4.95,-5.0,4.95,-5.0,0.00
2,Argentina,Latin America and Caribbean,6.26,6.14,-1.9,6.08,-2.9,-0.06
3,Armenia,Commonwealth of Independent States,5.38,4.99,-7.2,4.91,-8.7,-0.08
4,Australia,North America and ANZ,7.04,7.00,-0.6,6.90,-2.0,-0.10
5,Austria,Western Europe,7.00,6.71,-4.1,6.68,-4.6,-0.03
...,...,...,...,...,...,...,...,...
108,United States,North America and ANZ,6.69,6.13,-8.4,6.06,-9.4,-0.07
109,Uruguay,Latin America and Caribbean,6.67,6.33,-5.1,6.30,-5.5,-0.03
110,Uzbekistan,Commonwealth of Independent States,6.02,5.44,-9.6,5.19,-13.8,-0.25
112,Vietnam,Southeast Asia,6.27,5.25,-16.3,5.30,-15.5,0.05


In [22]:
# Ajout d'une colonne pour représenter les différences de pourcentages.
df_2022['% variations écarts'] = df_2022['Écart en % sans continents']-df_2022['Écart en % avec continents']
df_2022

,Country name,Regional indicator,Life Ladder,Prédictions avec continents,Écart en % avec continents,Prédictions sans continents,Écart en % sans continents,Différences valeurs écarts,% variations écarts
1,Albania,Central and Eastern Europe,5.21,4.95,-5.0,4.95,-5.0,0.00,0.0
2,Argentina,Latin America and Caribbean,6.26,6.14,-1.9,6.08,-2.9,-0.06,-1.0
3,Armenia,Commonwealth of Independent States,5.38,4.99,-7.2,4.91,-8.7,-0.08,-1.5
4,Australia,North America and ANZ,7.04,7.00,-0.6,6.90,-2.0,-0.10,-1.4
5,Austria,Western Europe,7.00,6.71,-4.1,6.68,-4.6,-0.03,-0.5
...,...,...,...,...,...,...,...,...,...
108,United States,North America and ANZ,6.69,6.13,-8.4,6.06,-9.4,-0.07,-1.0
109,Uruguay,Latin America and Caribbean,6.67,6.33,-5.1,6.30,-5.5,-0.03,-0.4
110,Uzbekistan,Commonwealth of Independent States,6.02,5.44,-9.6,5.19,-13.8,-0.25,-4.2
112,Vietnam,Southeast Asia,6.27,5.25,-16.3,5.30,-15.5,0.05,0.8


In [23]:
df_2022.describe()

,Life Ladder,Prédictions avec continents,Écart en % avec continents,Prédictions sans continents,Écart en % sans continents,Différences valeurs écarts,% variations écarts
count,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000
mean,5.591359,5.412039,-1.621359,5.346602,-2.754369,-0.065437,-1.133010
std,1.229449,1.052293,12.860150,1.049304,13.260858,0.150995,2.651207
min,2.350000,3.580000,-25.100000,3.410000,-25.600000,-0.580000,-10.500000
25%,4.675000,4.490000,-8.900000,4.415000,-9.100000,-0.105000,-1.900000
50%,5.890000,5.430000,-3.600000,5.300000,-4.500000,-0.030000,-0.500000
75%,6.465000,6.155000,1.300000,6.140000,1.450000,0.015000,0.300000
max,7.730000,7.510000,70.600000,7.430000,71.100000,0.400000,7.000000


In [24]:
# On constate que les prédictions sont moins précises si l'on enlève les données relatives aux continents.

In [25]:
# Séparation des écarts en positifs et négatifs
ecarts_positifs = df_2022[df_2022['Écart en % avec continents'] > 0]['Écart en % avec continents']
ecarts_negatifs = df_2022[df_2022['Écart en % avec continents'] < 0]['Écart en % avec continents']

# Calcul de la moyenne et de la médiane pour les écarts positifs
ecart_positif_moyen = ecarts_positifs.mean()
ecart_positif_median = ecarts_positifs.median()

# Calcul de la moyenne et de la médiane pour les écarts négatifs
ecart_negatif_moyen = ecarts_negatifs.mean()
ecart_negatif_median = ecarts_negatifs.median()

# Affichage des résultats
print("Écart négatif moyen:", ecart_negatif_moyen)
print("Écart positif moyen:", ecart_positif_moyen)
print("Écart négatif médian:", ecart_negatif_median)
print("Écart positif médian:", ecart_positif_median)

Écart négatif moyen: -7.507142857142857
Écart positif moyen: 11.203125
Écart négatif médian: -5.949999999999999
Écart positif médian: 5.45


In [26]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Calcul de l'erreur quadratique moyenne (MSE) et de la racine carrée de l'erreur quadratique moyenne (RMSE)
# avzec les continents.
mse_continents = mean_squared_error(y_test2022, y_predic2022_continents)
rmse_continents = np.sqrt(mse_continents)

# Affichage des résultats
print(f"Erreur quadratique moyenne avec les continents (MSE) : {mse_continents}")
print(f"Racine carrée de l'erreur quadratique moyenne avec les continents (RMSE) : {rmse_continents}")

# Calcul de l'erreur quadratique moyenne (MSE) et de la racine carrée de l'erreur quadratique moyenne (RMSE) 
# sans les continents.
mse_no_continents = mean_squared_error(y_test2022, y_predic2022_no_continents)
rmse_no_continents = np.sqrt(mse_no_continents)

# Affichage des résultats
print(f"Erreur quadratique moyenne sans les continents (MSE) : {mse_no_continents}")
print(f"Racine carrée de l'erreur quadratique moyenne sans les continents (RMSE) : {rmse_no_continents}")

Erreur quadratique moyenne avec les continents (MSE) : 0.3039135922330096
Racine carrée de l'erreur quadratique moyenne avec les continents (RMSE) : 0.5512835860362701
Erreur quadratique moyenne sans les continents (MSE) : 0.38762427184466014
Racine carrée de l'erreur quadratique moyenne sans les continents (RMSE) : 0.6225947894454789


In [27]:
# On constate que la RMSE augmente sans les continents. Le modèles est donc bien moins performant.

In [28]:
from joblib import dump, load

dump(rf_model_continents, '../models/happinessmeter.joblib')

['../models/happinessmeter.joblib']